# Under the hood.

## 1. Runtime / Kernels.

### 1.1. Pyodide vs Python.

The notebooks are designed to work in both conventional Python runtime (e.g. in JupyterLab, Colab), and in [Pyodide](https://pyodide.org/en/stable/) runtime kernel available with no setup on the web. The [JupyterLite](https://jupyterlite.readthedocs.io/en/stable/) framework is used for Pyodide.

### 1.2. Installing Packages.

The packages are installed based on the configuration(s) from the [config.yaml](../../config.yml) file available either (1) globally for all notebooks or (2) specific to a certain notebook by its name.

In Pyodide, the packages are installed as:

```python
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb")
```

where `create_interface_with_min_strain_zsl.ipynb` is the name of the current notebook, and `"../../config.yml"` is the relative path to the config file.


## 2. Data Exchange.

### 2.1. Pass Data to and from the Outside Runtime.
The data is passed in *from* and *back to* the outside runtime (e.g. web application) using the `get_data` and `set_data` [helpers](../../utils/jupyterlite.py). The `get_data` function is used to retrieve the data from the outside runtime, while the `set_data` function is used to send the data back to the outside runtime.

To get data, use `get_materials` function, a wrapper around `get_data('materials')

```python
from utils.jupyterlite import get_materials
materials = get_materials(globals())
```

Similarly, to pass data to the outside runtime, `set_materials` helper can be used:

```python
from utils.jupyterlite import set_materials
set_materials(interface)
```

where `interface` is the desired structure to be passed outside.

### 2.2. Uploads Folder.
The **"uploads"** folder is used to exchange files between different notebooks when a user uploads a file, or when the notebook generates a file as part of its output.

### 2.3. Passing Data between notebooks.
When running JupyterLab (Python) locally, the files only get stored in the "uploads" folder and are not sent to the outside runtime. In JupyterLite (Pyodide) environment, the files are sent to the outside runtime and stored in the "uploads" folder.


## 3. Specific Considerations.

### 3.1. Interface creation.

The following conventions are used in the interface creation examples:

3. We assume that two input materials are either in bulk form (e.g. Ni crystal) or layered (e.g. graphene).
1. We construct the interface along the Z-axis. The material corresponding to the bottom of the interface is referred to as the "**substrate**", and the top - as the "**film**".